* Câu 1: Hãy viết câu lệnh SQL để tính sự tương quan giữa A và B theo công thức sau:

$$
r_{AB} = \frac{n \sum (a_i b_i) - (\sum a_i \sum b_i)}
{\sqrt{n \sum a_i^2 - (\sum a_i)^2} \sqrt{n \sum b_i^2 - (\sum b_i)^2}}.
$$


* import thư viện : 

In [338]:
import sqlite3 
import math

* Kết nối SQLite3 với Python : 

In [339]:
conn = sqlite3.connect("database.db")
cursor = conn.cursor()


* Tạo bảng 

In [340]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS data_table (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    A REAL,
    B REAL
);
""")
# Lưu thay đổi vào database
conn.commit()


*  Chèn dữ liệu mẫu :

In [341]:
# Chèn dữ liệu mẫu 
cursor.execute("SELECT COUNT(*) FROM data_table")
if cursor.fetchone()[0] == 0:
    sample_data = [(2, 5), (3, 6), (4, 8), (5, 9), (6, 11)]
    cursor.executemany("INSERT INTO data_table (A, B) VALUES (?, ?);", sample_data)
    conn.commit()

* Hiển thị dữ liệu : 


In [342]:
# In dữ liệu từ bảng : 
cursor.execute("""
    SELECT * 
    FROM data_table
""")
du_lieu = cursor.fetchall()

# In kết quả ra màn hình
for dong in du_lieu:
    print(dong)


(1, 2.0, 5.0)
(2, 3.0, 6.0)
(3, 4.0, 8.0)
(4, 5.0, 9.0)
(5, 6.0, 11.0)


* Truy vấn dữ liệu cần để tính hệ số tương quan : 

In [343]:
cursor.execute("""
SELECT 
    COUNT(*) AS n,
    SUM(A) AS sum_A,
    SUM(B) AS sum_B,
    SUM(A * B) AS sum_AB,
    SUM(A * A) AS sum_A2,
    SUM(B * B) AS sum_B2
FROM data_table;
""")

n, sum_A, sum_B, sum_AB, sum_A2, sum_B2 = cursor.fetchone()


* Tính hệ số tương quan :


In [344]:
numerator = n * sum_AB - sum_A * sum_B
denominator = math.sqrt((n * sum_A2 - sum_A**2) * (n * sum_B2 - sum_B**2))

correlation = numerator / denominator if denominator != 0 else None


* Kết quả : 

In [345]:
print("Hệ số tương quan giữa A và B ", correlation )

# Đóng kết nối : 
conn.close()

Hệ số tương quan giữa A và B  0.993399267798783


Câu 2 : Một công ty oto đang kiểm tra 3 loại mẫu mới A, B và C trong 4 ngày, và chấm điểm theo thang từ 1 đến 10 điểm cho mỗi ngày với bảng sau. Liệu có sự khác biệt đáng kể giữa các mẫu dựa trên điểm số mà chúng nhận được trong 4 ngày thử nghiệm không Kết quả thử nghiệm phụ thuộc vào ngày hay phụ thuộc vào mẫu xe? Hãy chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm tra χ2.

|       | A   | B   | C   |
|-------|-----|-----|-----|
| Day 1 | 8   | 9   | 7   |
| Day 2 | 7.5 | 8.5 | 7   |
| Day 3 | 6   | 7   | 8   |
| Day 4 | 7   | 6   | 5   |


In [346]:
import sqlite3 
import pandas as pd
import scipy.stats as stats


* 1. Tạo kết nối SQLite :

In [347]:
conn = sqlite3.connect("car_test.db")
cursor = conn.cursor()

* Tạo bảng điểm :

In [348]:
# Xóa bảng : 
cursor.execute("DROP TABLE IF EXISTS car_scores;")


In [349]:
# Tạo bảng mới
cursor.execute("""
CREATE TABLE car_scores (
    Day TEXT,
    Model TEXT,
    Score REAL
);
""")

* Chèn dữu liệu vào bảng : 


In [350]:
cursor.execute("SELECT COUNT(*) FROM car_scores;")
if cursor.fetchone()[0] == 0:
    data = [
        ("Day 1", "A", 8), ("Day 1", "B", 9), ("Day 1", "C", 7),
        ("Day 2", "A", 7.5), ("Day 2", "B", 8.5), ("Day 2", "C", 7),
        ("Day 3", "A", 6), ("Day 3", "B", 7), ("Day 3", "C", 8),
        ("Day 4", "A", 7), ("Day 4", "B", 6), ("Day 4", "C", 5),
    ]
    cursor.executemany("INSERT INTO car_scores (Day, Model, Score) VALUES (?, ?, ?);", data)
    conn.commit()


* Truy vấn dữ liệu ra DataFrame:


In [351]:

df = pd.read_sql_query("SELECT * FROM car_scores", conn)
df



,Day,Model,Score
0,Day 1,A,8.0
1,Day 1,B,9.0
2,Day 1,C,7.0
3,Day 2,A,7.5
4,Day 2,B,8.5
5,Day 2,C,7.0
6,Day 3,A,6.0
7,Day 3,B,7.0
8,Day 3,C,8.0
9,Day 4,A,7.0


* Phân loaị điểm số thành nhóm :


In [352]:
def score_category(score):
    if score < 7:
        return "Low"
    elif score <= 8 :
        return "Medium"
    else :
        return "Hight"
df["ScoreCategory"] = df["Score"].apply(score_category)
# Hiển thị kết quả : 
print("\n Kết quả phân loại điểm số thành nhóm : ")
print(df)




 Kết quả phân loại điểm số thành nhóm : 
      Day Model  Score ScoreCategory
0   Day 1     A    8.0        Medium
1   Day 1     B    9.0         Hight
2   Day 1     C    7.0        Medium
3   Day 2     A    7.5        Medium
4   Day 2     B    8.5         Hight
5   Day 2     C    7.0        Medium
6   Day 3     A    6.0           Low
7   Day 3     B    7.0        Medium
8   Day 3     C    8.0        Medium
9   Day 4     A    7.0        Medium
10  Day 4     B    6.0           Low
11  Day 4     C    5.0           Low


* Tạo bảng chéo & kiểm định χ² theo Model

In [353]:
contingency_model = pd.crosstab(df["Model"], df["ScoreCategory"])
chi2_model, p_model, dof_model, expected_model = stats.chi2_contingency(contingency_model)


*  Tạo bảng chéo & kiểm định χ² theo Day

In [354]:
contingency_day = pd.crosstab(df["Day"], df["ScoreCategory"])
chi2_day, p_day, dof_day, expected_day = stats.chi2_contingency(contingency_day)


* KẾT QUẢ : 


In [355]:
print("KẾT QUẢ KIỂM ĐỊNH χ²: ")
print("\nTheo MODEL:")
print(contingency_model)
print(f"Chi² = {chi2_model:.4f}, p = {p_model:.4f}")


KẾT QUẢ KIỂM ĐỊNH χ²: 

Theo MODEL:
ScoreCategory  Hight  Low  Medium
Model                            
A                  0    1       3
B                  2    1       1
C                  0    1       3
Chi² = 5.1429, p = 0.2730


* p ≥ 0.05 :  không thấy sự khác biệt rõ ràng về điểm số giữa A, B, C → có thể giống nhau về chất lượng.

In [356]:

print("\nTheo NGÀY:")
print(contingency_day)
print(f"Chi² = {chi2_day:.4f}, p = {p_day:.4f}")



Theo NGÀY:
ScoreCategory  Hight  Low  Medium
Day                              
Day 1              1    0       2
Day 2              1    0       2
Day 3              0    1       2
Day 4              0    2       1
Chi² = 6.0952, p = 0.4126


* p ≥ 0.05 : điểm số ổn định giữa các ngày, ngày không ảnh hưởng nhiều đến kết quả.



In [357]:
# Đóng kết nối
conn.close()

* Câu 3 :  Bảng flights(departure_time,…) chứa các giá trị thời gian dưới dạng số nguyên (ví dụ: 830 cho 8:30 AM, 1445 cho 2:45 PM). Hãy chuyển đổi các giá trị này thành định dạng thời gian.

In [358]:
import sqlite3 


* Kết nối đến cơ sở dữ liệu SQLite

In [359]:
conn = sqlite3.connect('flights.db')
cursor = conn.cursor()


* Tạo bảng flights : 

In [360]:
# Tạo bảng flights
cursor.execute('''
CREATE TABLE IF NOT EXISTS flights (
    flight_id INTEGER PRIMARY KEY,
    departure_time INTEGER
)
''')

* Chèn dữ liệu mẫu : 

In [361]:
cursor.executemany('''
INSERT INTO flights (departure_time) VALUES (?)
''', [(830,), (1445,), (2359,), (1200,)])

# Lưu thay đổi
conn.commit()


* in dữ liệu


In [362]:
# Truy vấn và in dữ liệu
cursor.execute("SELECT * FROM flights")
flights_data = cursor.fetchall()

print("Dữ liệu bảng flights:")
for row in flights_data:
    print(row)

Dữ liệu bảng flights:
(1, 830)
(2, 1445)
(3, 2359)
(4, 1200)
(5, 830)
(6, 1445)
(7, 2359)
(8, 1200)
(9, 830)
(10, 1445)
(11, 2359)
(12, 1200)
(13, 830)
(14, 1445)
(15, 2359)
(16, 1200)
(17, 830)
(18, 1445)
(19, 2359)
(20, 1200)
(21, 830)
(22, 1445)
(23, 2359)
(24, 1200)
(25, 830)
(26, 1445)
(27, 2359)
(28, 1200)
(29, 830)
(30, 1445)
(31, 2359)
(32, 1200)
(33, 830)
(34, 1445)
(35, 2359)
(36, 1200)
(37, 830)
(38, 1445)
(39, 2359)
(40, 1200)
(41, 830)
(42, 1445)
(43, 2359)
(44, 1200)
(45, 830)
(46, 1445)
(47, 2359)
(48, 1200)
(49, 830)
(50, 1445)
(51, 2359)
(52, 1200)


* Truy vẫn và chuyển đổi dữ liệu thành định dạng thời gian và in kết quả :

In [363]:
# Truy vấn và chuyển đổi thời gian
cursor.execute('''
SELECT
    departure_time,
    -- Tách giờ và phút
    departure_time / 100 AS hour,
    departure_time % 100 AS minute,
    
    -- Chuyển giờ thành định dạng 12 giờ và xác định AM/PM
    CASE
        WHEN departure_time / 100 > 12 THEN departure_time / 100 - 12
        WHEN departure_time / 100 = 0 THEN 12
        ELSE departure_time / 100
    END AS hour_12,
    
    CASE
        WHEN departure_time / 100 >= 12 THEN 'PM'
        ELSE 'AM'
    END AS period,
    
    -- Định dạng thời gian
    printf('%02d:%02d %s', 
        CASE
            WHEN departure_time / 100 > 12 THEN departure_time / 100 - 12
            WHEN departure_time / 100 = 0 THEN 12
            ELSE departure_time / 100
        END, 
        departure_time % 100, 
        CASE
            WHEN departure_time / 100 >= 12 THEN 'PM'
            ELSE 'AM'
        END
    ) AS formatted_time
FROM flights
''')

# Lấy kết quả
rows = cursor.fetchall()

# In kết quả ra màn hình
for row in rows:
    print(f"departure_time: {row[0]}, formatted_time: {row[5]}")


departure_time: 830, formatted_time: 08:30 AM
departure_time: 1445, formatted_time: 02:45 PM
departure_time: 2359, formatted_time: 11:59 PM
departure_time: 1200, formatted_time: 12:00 PM
departure_time: 830, formatted_time: 08:30 AM
departure_time: 1445, formatted_time: 02:45 PM
departure_time: 2359, formatted_time: 11:59 PM
departure_time: 1200, formatted_time: 12:00 PM
departure_time: 830, formatted_time: 08:30 AM
departure_time: 1445, formatted_time: 02:45 PM
departure_time: 2359, formatted_time: 11:59 PM
departure_time: 1200, formatted_time: 12:00 PM
departure_time: 830, formatted_time: 08:30 AM
departure_time: 1445, formatted_time: 02:45 PM
departure_time: 2359, formatted_time: 11:59 PM
departure_time: 1200, formatted_time: 12:00 PM
departure_time: 830, formatted_time: 08:30 AM
departure_time: 1445, formatted_time: 02:45 PM
departure_time: 2359, formatted_time: 11:59 PM
departure_time: 1200, formatted_time: 12:00 PM
departure_time: 830, formatted_time: 08:30 AM
departure_time: 144

In [364]:
# Đóng kết nối
conn.close()

* Câu 4: Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD. Một quy tắc chung là xem xét các giá
trị ngoại lệ lớn hơn 1,5 lần so với giá trị MAD, trong đó x là số độ lệch chuẩn mà ta coi là có ý nghĩa.

In [365]:
import sqlite3 

* Kết nối tới cơ sở dữ liệu SQLite 

In [366]:
conn = sqlite3.connect("sales.db")

* Tạo đối tượng con trỏ để thực thi các câu lệnh SQL

In [367]:
cursor = conn.cursor()

* Tạo bảng sales_data

In [368]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS sales_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    sales_value INTEGER
)
''')

# Lưu thay đổi
conn.commit()

* Chèn dữ liệu mẫu vào bảng sales_data

In [369]:
# Chèn dữ liệu mẫu vào bảng sales_data
cursor.executemany('''
INSERT INTO sales_data (sales_value) VALUES (?)
''', [(1200,), (1500,), (3000,), (1100,), (1600,), (5000,)])

# Lưu thay đổi
conn.commit()

* Hiển thị dữ liệu : 

In [370]:
# In dữ liệu từ bảng : 
cursor.execute("""
    SELECT * 
    FROM sales_data  """)
deviations = cursor.fetchall()
deviations

[(1, 1200),
 (2, 1500),
 (3, 3000),
 (4, 1100),
 (5, 1600),
 (6, 5000),
 (7, 1200),
 (8, 1500),
 (9, 3000),
 (10, 1100),
 (11, 1600),
 (12, 5000),
 (13, 1200),
 (14, 1500),
 (15, 3000),
 (16, 1100),
 (17, 1600),
 (18, 5000),
 (19, 1200),
 (20, 1500),
 (21, 3000),
 (22, 1100),
 (23, 1600),
 (24, 5000),
 (25, 1200),
 (26, 1500),
 (27, 3000),
 (28, 1100),
 (29, 1600),
 (30, 5000),
 (31, 1200),
 (32, 1500),
 (33, 3000),
 (34, 1100),
 (35, 1600),
 (36, 5000),
 (37, 1200),
 (38, 1500),
 (39, 3000),
 (40, 1100),
 (41, 1600),
 (42, 5000),
 (43, 1200),
 (44, 1500),
 (45, 3000),
 (46, 1100),
 (47, 1600),
 (48, 5000),
 (49, 1200),
 (50, 1500),
 (51, 3000),
 (52, 1100),
 (53, 1600),
 (54, 5000),
 (55, 1200),
 (56, 1500),
 (57, 3000),
 (58, 1100),
 (59, 1600),
 (60, 5000),
 (61, 1200),
 (62, 1500),
 (63, 3000),
 (64, 1100),
 (65, 1600),
 (66, 5000),
 (67, 1200),
 (68, 1500),
 (69, 3000),
 (70, 1100),
 (71, 1600),
 (72, 5000)]

* Tính trung vị (Median)

In [371]:
# Tính trung vị (Median)
cursor.execute('''
WITH ordered_sales AS (
    SELECT sales_value, ROW_NUMBER() OVER (ORDER BY sales_value) AS row_num, COUNT(*) OVER () AS total_rows
    FROM sales_data
)
SELECT sales_value
FROM ordered_sales
WHERE row_num = (total_rows + 1) / 2
LIMIT 1;
''')

# Lấy trung vị
median_value = cursor.fetchone()[0]
print(f"Median Value: {median_value}")

Median Value: 1500


* Tính độ lệch tuyệt đối từ trung vị (MAD)

In [372]:
# Tính độ lệch tuyệt đối từ trung vị (MAD)
cursor.execute('''
SELECT sales_value, ABS(sales_value - ?) AS deviation
FROM sales_data;
''', (median_value,))

# Lấy các độ lệch tuyệt đối
deviations = cursor.fetchall()
print("Deviations from median:")
for row in deviations:
    print(f"Sales Value: {row[0]}, Deviation: {row[1]}")


Deviations from median:
Sales Value: 1200, Deviation: 300
Sales Value: 1500, Deviation: 0
Sales Value: 3000, Deviation: 1500
Sales Value: 1100, Deviation: 400
Sales Value: 1600, Deviation: 100
Sales Value: 5000, Deviation: 3500
Sales Value: 1200, Deviation: 300
Sales Value: 1500, Deviation: 0
Sales Value: 3000, Deviation: 1500
Sales Value: 1100, Deviation: 400
Sales Value: 1600, Deviation: 100
Sales Value: 5000, Deviation: 3500
Sales Value: 1200, Deviation: 300
Sales Value: 1500, Deviation: 0
Sales Value: 3000, Deviation: 1500
Sales Value: 1100, Deviation: 400
Sales Value: 1600, Deviation: 100
Sales Value: 5000, Deviation: 3500
Sales Value: 1200, Deviation: 300
Sales Value: 1500, Deviation: 0
Sales Value: 3000, Deviation: 1500
Sales Value: 1100, Deviation: 400
Sales Value: 1600, Deviation: 100
Sales Value: 5000, Deviation: 3500
Sales Value: 1200, Deviation: 300
Sales Value: 1500, Deviation: 0
Sales Value: 3000, Deviation: 1500
Sales Value: 1100, Deviation: 400
Sales Value: 1600, Deviat

* Tính giá trih MAD : 


In [373]:
# Tính MAD 
cursor.execute('''
WITH deviation_data AS (
    SELECT ABS(sales_value - ?) AS deviation
    FROM sales_data
)
SELECT deviation
FROM deviation_data
ORDER BY deviation
LIMIT 1 OFFSET (SELECT (COUNT(*) - 1) / 2 FROM deviation_data);
''', (median_value,))

# Lấy giá trị MAD
mad_value = cursor.fetchone()[0]
print(f"Giá trị MAD : {mad_value}")


Giá trị MAD : 300


* Tìm các giá trị ngoại lệ : 


In [374]:
# Tìm các giá trị ngoại lệ (deviation > 1.5 * MAD)
cursor.execute('''
WITH deviation_data AS (
    SELECT sales_value, ABS(sales_value - ?) AS deviation
    FROM sales_data
)
SELECT sales_value
FROM deviation_data
WHERE deviation > 1.5 * ?
''', (median_value, mad_value))

# Lấy các giá trị ngoại lệ
outliers = cursor.fetchall()
print("Outliers:")
for row in outliers:
    print(f"Outlier Sales Value: {row[0]}")


Outliers:
Outlier Sales Value: 3000
Outlier Sales Value: 5000
Outlier Sales Value: 3000
Outlier Sales Value: 5000
Outlier Sales Value: 3000
Outlier Sales Value: 5000
Outlier Sales Value: 3000
Outlier Sales Value: 5000
Outlier Sales Value: 3000
Outlier Sales Value: 5000
Outlier Sales Value: 3000
Outlier Sales Value: 5000
Outlier Sales Value: 3000
Outlier Sales Value: 5000
Outlier Sales Value: 3000
Outlier Sales Value: 5000
Outlier Sales Value: 3000
Outlier Sales Value: 5000
Outlier Sales Value: 3000
Outlier Sales Value: 5000
Outlier Sales Value: 3000
Outlier Sales Value: 5000
Outlier Sales Value: 3000
Outlier Sales Value: 5000


In [375]:
# Đóng kết nối
conn.close() 


* Câu 5 : Hãy xác định liệu hai người trong bảng Patient(last_name, weight, height) có phải là một người hay không bằng cách sử dụng khoảng cách kết hợp Boolean trên “last_name” và “weight”. 

* import thư viện : 

In [ ]:

import sqlite3 
import pandas as pd 


* Kết nối SQLite : 

In [377]:
# Kết nối SQLite
conn = sqlite3.connect("du_lieu.db")
cursor = conn.cursor()



*  Tạo bảng Patient : 


In [379]:
cursor.execute("""
CREATE TABLE Patient (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    height REAL,
    weight REAL
)
""")



* Tạo dữ liệu mẫu :


In [380]:
sample_data = [
    (1, "Nguyen", 170.5, 65),
    (2, "Tran", 165.2, 70.3),
    (3, "Nguyen", 180.1, 75.5),
    (4, "Le", 160, 55.2),
    (5, "Nguyen", 170.4, None)  # Có NULL
]

cursor.executemany("INSERT INTO Patient VALUES (?, ?, ?, ?)", sample_data)
conn.commit()

* Kết quả bảng đã tạo : 


In [381]:
# Truy vấn tất cả dữ liệu từ bảng Patient
cursor.execute("SELECT * FROM Patient")
patients = cursor.fetchall()

# In kết quả
print("Dữ liệu bảng Patient:")
for row in patients:
    print(row)


Dữ liệu bảng Patient:
(1, 'Nguyen', 170.5, 65.0)
(2, 'Tran', 165.2, 70.3)
(3, 'Nguyen', 180.1, 75.5)
(4, 'Le', 160.0, 55.2)
(5, 'Nguyen', 170.4, None)


* Truy vấn để so sánh từng cặp bệnh nhân : 

In [382]:
query = '''
SELECT 
    p1.id AS id1, p2.id AS id2,
    p1.last_name AS name1, p2.last_name AS name2,
    p1.weight AS weight1, p2.weight AS weight2,
    CASE
        WHEN p1.last_name = p2.last_name 
             AND p1.weight IS NOT NULL AND p2.weight IS NOT NULL
             AND ABS(p1.weight - p2.weight) < 1.0
        THEN 1
        ELSE 0
    END AS same_person
FROM Patient p1
JOIN Patient p2 ON p1.id < p2.id
'''

df_result = pd.read_sql_query(query, conn)
print(df_result)

   id1  id2   name1   name2  weight1  weight2  same_person
0    1    2  Nguyen    Tran     65.0     70.3            0
1    1    3  Nguyen  Nguyen     65.0     75.5            0
2    1    4  Nguyen      Le     65.0     55.2            0
3    1    5  Nguyen  Nguyen     65.0      NaN            0
4    2    3    Tran  Nguyen     70.3     75.5            0
5    2    4    Tran      Le     70.3     55.2            0
6    2    5    Tran  Nguyen     70.3      NaN            0
7    3    4  Nguyen      Le     75.5     55.2            0
8    3    5  Nguyen  Nguyen     75.5      NaN            0
9    4    5      Le  Nguyen     55.2      NaN            0


Nhận xét kết quả trên : 
* Cột same_person đều bằng 0 → nghĩa là không có cặp nào đủ điều kiện để được coi là cùng một người theo tiêu chí bạn đặt ra.